In [120]:
import numpy as np
import time
from selenium import webdriver
from typing import List
import pandas as pd


In [121]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

chrome_options = Options()
#chrome_options.add_argument("--headless")

In [122]:
BASE_URL = "https://www.eventiesagre.it/elenco.html"
chromedriver = "/home/alesssandro/sw/chromedriver"
driver = webdriver.Chrome (executable_path="/home/alesssandro/sw/chromedriver")
browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chrome_options)
browser.get(BASE_URL)
wait = WebDriverWait(browser, 10)

<ipython-input-122-575b99e7ea26>:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chrome_options)


In [123]:
events_list = {'Title': [], 'Region': [], 'Date from-to': []}#dictionary of events 
date_path = "//div[@class='DX']/div[@class='testo']/div[1]"#path to locate date of each events on the Website
title_path = "//div[@class='DX']/div[@class='testo']/h3[@class='titolo']"#path to locate title of each events on the Website
region_path = "//div[@class='DX']/div[@class='testo']/span[@class='grassetto'][1]"#path to locate region of each events on the Website

In [124]:
def waitUntil():#wait until all 14 events have been loaded on the Website and then it goes to the next page by clicking on "Avanti" link
    wait.until(lambda brow: len(brow.find_elements_by_xpath(date_path)) == 14)
    wait.until(lambda brow: len(brow.find_elements_by_xpath(title_path)) == 14)
    wait.until(lambda brow: len(brow.find_elements_by_xpath(region_path)) == 14)
    time.sleep(2)
    browser.find_element_by_link_text("Avanti").click();

In [125]:
def change_date_value():# cleaning date value
    cleaned_list = []
    values = events_list["Date from-to"]

    for i in values:
        temp = i.replace("Dal", "").replace('Al','-').replace(" ", "").replace('Il','')
        cleaned_list.append(temp)
    events_list["Date from-to"].clear()
    for n in cleaned_list:
        events_list["Date from-to"].append(n)

In [126]:
def get_values(list_name, path_name, dictionary_key):#get values of events and append them to "events_list" dictionary
    for value in browser.find_elements_by_xpath(path_name):
        list_name.append(value.text)  
    del list_name[0:4]#list is filled with 14 elements, the firs 4 are sponsor, this line only keep the following events before append them 
    
    for line in list_name:
        events_list["{}".format(dictionary_key)].append(line)
    

In [127]:
def get_data():# get all events data thanks to "get_values"  function at the same time
    titles_list = []
    dates_list = []
    regions_list = []
    get_values(titles_list, title_path, "Title")
    get_values(dates_list, date_path, "Date from-to")
    get_values(regions_list, region_path, "Region")


In [128]:
for prova in range(2):
#while (browser.find_element_by_link_text("Avanti")):#loop which iterates data collection until "Avanti" link is clickable
    get_data()                                      # and other pages to iterate are still available
    waitUntil()
change_date_value()

In [129]:
df = pd.DataFrame(data = events_list)
df.to_csv('./report.csv', index=None)